In [1]:
import arcpy
from archaic import FeatureClass

arcpy.env.workspace = ".data/world.geodatabase"  # type: ignore

class City:
    objectid: int
    city_name: str
    pop: int
    shape: arcpy.PointGeometry

feature_class = FeatureClass[City]("cities")

for city in feature_class.read(wkid=4326):
    print(city.city_name, city.shape.WKT)


Cuiaba POINT (-56.09300367487964 -15.615000071396139)
Brasilia POINT (-47.897747657328466 -15.792110942665829)
Goiania POINT (-49.25500160334127 -16.727003924891164)
Campo Grande POINT (-54.615995658972452 -20.450997053571143)
Pedro Juan Caballero POINT (-55.747997716121688 -22.533003066738104)
Salto del Guaira POINT (-54.283301684325238 -24.049998057864059)
Puerto Presidente Stroessner POINT (-54.622997604156069 -25.526996972632162)
Encarnacion POINT (-55.851002689022835 -27.377001018582121)
Posadas POINT (-55.906001679333997 -27.397998082145705)
Puerto Maldonado POINT (-69.191999562829494 -12.60299707605418)
Cuzco POINT (-71.987003670383046 -13.526001049608681)
Abancay POINT (-72.883997584820634 -13.642001950029908)
Puno POINT (-70.034003587195741 -15.845003988743031)
Arequipa POINT (-71.522999570473758 -16.398000070047146)
La Paz POINT (-68.146244609601013 -16.49900699307824)
Mata-Utu POINT (-176.15156363635271 -13.273299062201886)
Apia POINT (-171.73949359035689 -13.837022952946882